In [1]:
import math
import re
import sys
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# load pre-trained Word2Vec model
model = api.load('word2vec-google-news-300')

In [4]:
def convert_matrix_to_dict(wordVecMat, wordList):
    wordVecs = {}
    for i, word in enumerate(wordList):
        wordVecs[word] = wordVecMat[i]
    return wordVecs

def vectorize_list(corpus):
    corpus_vecs = [model[word] for word in corpus]
    return corpus_vecs

In [106]:
# Get the same input format as the real corpus
toy_corpus = ["cat", "tiger", "computer", "keyboard", "plane", "car", "doctor", "nurse", "love", "sex"]
toy_corpus_vecs = vectorize_list(toy_corpus)
toy_wordVecs = convert_matrix_to_dict(toy_corpus_vecs, toy_corpus)

In [107]:
# Different calculation tries
def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_product = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    similarity = dot_product / norm_product
    return similarity

def generate_cosine_similarity_matrix(vectors):
    num_vectors = len(vectors)
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            similarity = calculate_cosine_similarity(vectors[i], vectors[j])
            similarity_matrix[i, j] = similarity
    return similarity_matrix

def print_vec_similarities(corpus, vectorized_corpus):
    for i, vec1 in enumerate(vectorized_corpus):
        word1 = corpus[i]
        print(f'Similarities with "{word1}":')
        for j, vec2 in enumerate(vectorized_corpus):
            similarity = calculate_cosine_similarity(vec1, vec2)
            print(f'  - "{corpus[j]}": {similarity:.4f}')
        print()

def print_similarity_difference(similarity_matrix, retrofitted_similarity_matrix):
    difference = np.abs(similarity_matrix - retrofitted_similarity_matrix)
    print("Similarity Difference Matrix:")
    print(difference)

def cosine_similarity_matrix(matrix1, matrix2):
    dot_product = np.sum(matrix1 * matrix2)
    norm_matrix1 = np.linalg.norm(matrix1)
    norm_matrix2 = np.linalg.norm(matrix2)
    cosine_similarity = dot_product / (norm_matrix1 * norm_matrix2)
    return cosine_similarity

In [108]:
similarity_matrix = generate_cosine_similarity_matrix(toy_corpus_vecs)
print_vec_similarities(toy_corpus, similarity_matrix)

Similarities with "cat":
  - "cat": 1.0000
  - "tiger": 0.8251
  - "computer": 0.4368
  - "keyboard": 0.4233
  - "plane": 0.4638
  - "car": 0.5096
  - "doctor": 0.3656
  - "nurse": 0.3922
  - "love": 0.3505
  - "sex": 0.4119

Similarities with "tiger":
  - "cat": 0.8251
  - "tiger": 1.0000
  - "computer": 0.2903
  - "keyboard": 0.2601
  - "plane": 0.4112
  - "car": 0.4304
  - "doctor": 0.2921
  - "nurse": 0.3147
  - "love": 0.2867
  - "sex": 0.4659

Similarities with "computer":
  - "cat": 0.4368
  - "tiger": 0.2903
  - "computer": 1.0000
  - "keyboard": 0.7100
  - "plane": 0.4626
  - "car": 0.5466
  - "doctor": 0.4371
  - "nurse": 0.4812
  - "love": 0.2559
  - "sex": 0.4320

Similarities with "keyboard":
  - "cat": 0.4233
  - "tiger": 0.2601
  - "computer": 0.7100
  - "keyboard": 1.0000
  - "plane": 0.3304
  - "car": 0.4082
  - "doctor": 0.2954
  - "nurse": 0.3373
  - "love": 0.3576
  - "sex": 0.3189

Similarities with "plane":
  - "cat": 0.4638
  - "tiger": 0.4112
  - "computer": 0.4

In [109]:
toy_wordVecs # same format as the embeddings when retrieved

{'cat': array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
         0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
         0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
        -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
         0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
         0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
        -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
         0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
         0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
        -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
        -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
        -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
         0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
        -0.08251953,  0.05957031,  0.1020507

In [111]:
def get_wordnet_lexicon(target_words, relation_types):
    lexicon = {}

    for word in target_words:
        related_words = []
        word_synsets = wordnet.synsets(word)
        
        # Skip word if no synsets found
        if not word_synsets:
            continue

        for syn in word_synsets:
            for lemma in syn.lemmas():
                if lemma.name() != word:
                    if "synonyms" in relation_types:
                        related_words.append(lemma.name())
            if "antonyms" in relation_types:
                if syn.lemmas()[0].antonyms():
                    related_words.append(syn.lemmas()[0].antonyms()[0].name())
            if "hyponyms" in relation_types:
                for hypo in syn.hyponyms():
                    for lemma in hypo.lemmas():
                        related_words.append(lemma.name())
            if "hypernyms" in relation_types:
                for hyper in syn.hypernyms():
                    for lemma in hyper.lemmas():
                        related_words.append(lemma.name())
            if "meronyms" in relation_types:
                for part in syn.part_meronyms():
                    for lemma in part.lemmas():
                        related_words.append(lemma.name())
            if "holonyms" in relation_types:
                for whole in syn.part_holonyms():
                    for lemma in whole.lemmas():
                        related_words.append(lemma.name())
            if "homonyms" in relation_types:
                for lemma in syn.lemmas():
                    if lemma.name() != word:
                        homonyms = wordnet.lemmas(lemma.name())
                        for homonym in homonyms:
                            related_words.append(homonym.name())
        lexicon[word] = related_words
    return lexicon

In [112]:
def convert_embeddings_to_matrix(wordVecs):
    wordVecMat = np.array(list(wordVecs.values()))
    return wordVecMat

wordVecMat = convert_embeddings_to_matrix(toy_wordVecs)

In [114]:
# This outputs the same list as the initial toy_corpus but it is used to retrieve the keys from the gensim word embedding dict
def get_embeddings_words(wordVecs):
    wordList = list(wordVecs.keys()) 
    return wordList

wordList = get_embeddings_words(toy_wordVecs)

In [115]:
def create_neighbors_embedding_matrix(wordList, relation_type):
    # Retrieve synonyms for each word
    neighbors_dict = get_wordnet_lexicon(wordList, relation_type)
    
    # Compute average embedding
    average_embeddings = []
    for word in wordList:
        neighbors = neighbors_dict.get(word, [])
        embeddings = [
            model.get_vector(neighbor)
            for neighbor in neighbors
            if model.has_index_for(neighbor)
        ]
        if len(embeddings) > 0:
            average_embedding = np.sum(embeddings, axis=0) / len(embeddings)
        else:
            # Handle the case where a word has no embeddings for its synonyms
            average_embedding = np.zeros(model.vector_size)  
        average_embeddings.append(average_embedding)
    
    # Create the word embedding matrix
    neighbors_embedding_matrix = np.vstack(average_embeddings)
    return neighbors_embedding_matrix

neighbors_matrix = create_neighbors_embedding_matrix(wordList, "synonyms")

In [116]:
# Checking that our matrices have the same shapes 
print(type(neighbors_matrix))  
print(neighbors_matrix.shape)  
print(neighbors_matrix.ndim)   

<class 'numpy.ndarray'>
(10, 300)
2


In [173]:
print(type(wordVecMat))  
print(wordVecMat.shape)  
print(wordVecMat.ndim)   

<class 'numpy.ndarray'>
(10, 300)
2


In [118]:
# Checking the difference between the embedding of the first word and the averaged embedding of its synonyms
difference = toy_corpus_vecs[0] - neighbors_matrix[0]
print(difference)

[ 0.00676925  0.17245371 -0.32560504 -0.02995695  0.24918167  0.06591797
  0.07754347  0.02062197  0.12026186 -0.24899179  0.11663705 -0.43598995
 -0.0165247  -0.43618888  0.05141873 -0.2046328   0.09973597  0.0461245
 -0.4420053   0.08787028  0.26338252 -0.1518453   0.11536521 -0.14642108
 -0.04931188  0.32405486 -0.15808557  0.31547716  0.40427201 -0.007934
 -0.00195312 -0.20903128 -0.03433369 -0.0849519   0.01340795  0.15198545
 -0.07754234  0.04350902  0.00148463  0.24564164  0.00093107  0.03174506
 -0.14347048  0.13828702  0.08990253 -0.03038646  0.37447442 -0.05503337
  0.00611821  0.00164738 -0.15880896  0.13665545  0.34953252  0.14862174
 -0.0103189  -0.14141733  0.15832859 -0.30018898  0.35803392  0.18776052
  0.28351056  0.18667716 -0.01274052  0.19207764  0.04131345  0.04772498
  0.20808016  0.02882668 -0.10890771 -0.12867793  0.40634721 -0.03548855
 -0.15468343 -0.01318077 -0.11528298  0.38514766  0.11844098 -0.08239746
 -0.02005401 -0.06692618  0.08503554 -0.00346544  0.19

In [120]:
def retrofitting_wordVecs(wordVecMat, neighbors_mean_matrix, alpha, beta, nb_iter):
    # Create a deep copy of wordVecMat 
    newWordVecMat = np.copy(wordVecMat, order='K')
    updates = []
    
    for _ in range(nb_iter):
        # Calculate the number of neighbors for each word
        # numNeighbors = np.sum(neighbors_mean_matrix != 0, axis=1)
        
        # Update the word embeddings using retrofitting formula
        newWordVecMat = (alpha * newWordVecMat + beta * neighbors_mean_matrix) / (alpha + beta)

        # Calculate the updates
        update = newWordVecMat - wordVecMat
        updates.append(update)

        # Update the wordVecMat for the next iteration
        wordVecMat = newWordVecMat

        # Stoping criterion
        if np.linalg.norm(updates) < 1e-2:
            break 

    # Convert the matrix back to a dictionary of word vectors
    # retrofitted_wordVecs = dict(zip(wordList, wordVecMat))

    return newWordVecMat, updates

In [123]:
retrofitted_toy_vecs, updates = retrofitting_wordVecs(wordVecMat, neighbors_matrix, alpha=1, beta=1, nb_iter=10)

In [127]:
# Computing the similarities between the words of the toy corpus after the retrofitting ==> BUT THERE IS A CLEAR PROBLEM
retrofitted_similarity_matrix = generate_cosine_similarity_matrix(retrofitted_toy_vecs)
print_vec_similarities(toy_corpus, retrofitted_similarity_matrix)

Similarities with "cat":
  - "cat": 1.0000
  - "tiger": 0.5027
  - "computer": 0.5525
  - "keyboard": 0.5658
  - "plane": 0.7556
  - "car": 0.6173
  - "doctor": 0.7336
  - "nurse": 0.8918
  - "love": 0.9206
  - "sex": 0.6472

Similarities with "tiger":
  - "cat": 0.5027
  - "tiger": 1.0000
  - "computer": 0.3166
  - "keyboard": 0.4519
  - "plane": 0.3484
  - "car": 0.3352
  - "doctor": 0.3611
  - "nurse": 0.5076
  - "love": 0.5046
  - "sex": 0.4776

Similarities with "computer":
  - "cat": 0.5525
  - "tiger": 0.3166
  - "computer": 1.0000
  - "keyboard": 0.5768
  - "plane": 0.7585
  - "car": 0.5762
  - "doctor": 0.5343
  - "nurse": 0.4580
  - "love": 0.5712
  - "sex": 0.3318

Similarities with "keyboard":
  - "cat": 0.5658
  - "tiger": 0.4519
  - "computer": 0.5768
  - "keyboard": 1.0000
  - "plane": 0.6428
  - "car": 0.4836
  - "doctor": 0.4641
  - "nurse": 0.5095
  - "love": 0.6121
  - "sex": 0.3912

Similarities with "plane":
  - "cat": 0.7556
  - "tiger": 0.3484
  - "computer": 0.7

In [129]:
def print_vec_difference(wordList, similarity_matrix1, similarity_matrix2):
    for i, word in enumerate(wordList):
        print(f"\nSimilarities with \"{word}\":")
        for j, neighbor in enumerate(wordList):
            similarity1 = similarity_matrix1[i, j]
            similarity2 = similarity_matrix2[i, j]
            difference = similarity2 - similarity1  
            print(f"  - \"{neighbor}\": {similarity1:.4f} -> {similarity2:.4f} (Difference: {difference:.4f})")

print_vec_difference(toy_corpus, similarity_matrix, retrofitted_similarity_matrix)


Similarities with "cat":
  - "cat": 1.0000 -> 1.0000 (Difference: 0.0000)
  - "tiger": 0.5173 -> 0.2028 (Difference: -0.3145)
  - "computer": 0.1732 -> 0.1974 (Difference: 0.0241)
  - "keyboard": 0.1834 -> 0.2057 (Difference: 0.0223)
  - "plane": 0.1833 -> 0.3947 (Difference: 0.2114)
  - "car": 0.2153 -> 0.2636 (Difference: 0.0483)
  - "doctor": 0.1292 -> 0.3436 (Difference: 0.2144)
  - "nurse": 0.1594 -> 0.5487 (Difference: 0.3893)
  - "love": 0.1406 -> 0.5941 (Difference: 0.4535)
  - "sex": 0.1368 -> 0.2601 (Difference: 0.1233)

Similarities with "tiger":
  - "cat": 0.5173 -> 0.2028 (Difference: -0.3145)
  - "tiger": 1.0000 -> 1.0000 (Difference: 0.0000)
  - "computer": 0.0677 -> 0.0903 (Difference: 0.0226)
  - "keyboard": 0.0654 -> 0.1831 (Difference: 0.1177)
  - "plane": 0.1660 -> 0.0734 (Difference: -0.0926)
  - "car": 0.1672 -> 0.1005 (Difference: -0.0668)
  - "doctor": 0.0835 -> 0.0891 (Difference: 0.0056)
  - "nurse": 0.1111 -> 0.2028 (Difference: 0.0917)
  - "love": 0.0871 ->

In [131]:
# Calculate cosine similarity between wordVecMat and retrofitted_toy_vec
similarity_score = cosine_similarity_matrix(wordVecMat, retrofitted_toy_vecs)
print("Cosine Similarity:", similarity_score)

Cosine Similarity: 0.4249422167016215


In [132]:
def measure_embedding_updates(original_matrix, retrofitted_matrix):
    absolute_diff = np.abs(original_matrix - retrofitted_matrix)
    mean_absolute_diff = np.mean(absolute_diff)
    return mean_absolute_diff

update_measure = measure_embedding_updates(wordVecMat, retrofitted_toy_vecs)
print("Average embedding update:", update_measure)

Average embedding update: 0.12658860989283166


In [140]:
def spearman_measure_embedding_similarity(original_matrix, retrofitted_matrix):
    original_flat = original_matrix.flatten()
    retrofitted_flat = retrofitted_matrix.flatten()
    correlation, _ = spearmanr(original_flat, retrofitted_flat)
    return correlation

def pearson_measure_embedding_similarity(original_matrix, retrofitted_matrix):
    original_flat = original_matrix.flatten()
    retrofitted_flat = retrofitted_matrix.flatten()
    correlation, _ = pearsonr(original_flat, retrofitted_flat)
    return correlation

similarity_score = spearman_measure_embedding_similarity(wordVecMat, retrofitted_toy_vecs)
print("Spearman correlation score:", similarity_score)
similarity_score = pearson_measure_embedding_similarity(wordVecMat, retrofitted_toy_vecs)
print("Pearson correlation score:", similarity_score)

Spearman correlation score: 0.42055888661904023
Pearson correlation score: 0.4232245375810048


In [134]:
# Here we are trying to get better hyperparameters. But the values we get after these steps perform less well than in the following cell where we only check for the embedding's update

# Load human evaluation scores
eval_file_path = r"ws353_lexical_similarity.txt"
eval_scores = {}
with open(eval_file_path, 'r') as file:
    for line in file:
        word1, word2, score = line.strip().split('\t')
        eval_scores[(word1, word2)] = float(score)

# Find best values for hyperparameters
best_similarity_score = -1  
best_params = {}  
iteration_count = 0 # stopping criterion especially useful for the gensim file

for alpha in np.arange(0.1, 5.1, 0.2):
    for beta in np.arange(0.1, 5.1, 0.2):
        for nb_iter in range(1, 16):
            retrofitted_toy_vec, _ = retrofitting_wordVecs(wordVecMat, neighbors_matrix, alpha, beta, nb_iter)
            cosine_sim = cosine_similarity(wordVecMat, retrofitted_toy_vec)

            # Calculate Spearman correlation against human evaluation scores
            eval_scores_list = []
            cosine_sim_list = []
            for (word1, word2), score in eval_scores.items():
                if word1 in wordList and word2 in wordList:
                    word1_index = wordList.index(word1)
                    word2_index = wordList.index(word2)
                    eval_scores_list.append(score)
                    cosine_sim_list.append(cosine_sim[word1_index, word2_index])

            # Check if there are valid pairs for comparison
            if len(eval_scores_list) > 0 and len(cosine_sim_list) > 0:
                correlation, _ = spearmanr(eval_scores_list, cosine_sim_list)
                # print("alpha =", alpha, "beta =", beta, "nb_iter =", nb_iter, "correlation =", correlation)

                # Update best similarity score and parameters if improved
                if correlation > best_similarity_score:
                    best_similarity_score = correlation
                    best_params = {'alpha': alpha, 'beta': beta, 'nb_iter': nb_iter}

            iteration_count += 1
            if iteration_count >= 100:
                break
        if iteration_count >= 100:
            break
    if iteration_count >= 100:
        break

print("Best hyperparameters:", best_params)
print("Best Spearman correlation score:", best_similarity_score)

Best hyperparameters: {'alpha': 0.1, 'beta': 0.1, 'nb_iter': 1}
Best Spearman correlation score: 0.7142857142857143


In [135]:
for alpha in np.arange(0.1, 5.1, 0.2):
    for beta in np.arange(0.1, 5.1, 0.2):
        for nb_iter in range(1,16):
            retrofitted_toy_vec, _ = retrofitting_wordVecs(wordVecMat, neighbors_matrix, alpha, beta, nb_iter)
            embed_update = measure_embedding_updates(wordVecMat, retrofitted_toy_vec)
            # print(" alpha =", alpha, " beta=", beta, "nb_iter =", nb_iter, " similarity score =", similarity_score)
            if embed_update > best_embed_update:
                best_embed_update = embed_update
                best_params = {'alpha': alpha, 'beta': beta, 'nb_iter': nb_iter}

print("Best hyperparameters:", best_params)
print("Best embedding update:", best_embed_update)

Best hyperparameters: {'alpha': 0.1, 'beta': 1.1000000000000003, 'nb_iter': 15}
Best embedding update: 0.12671235242449622


In [136]:
# Performing the retrofitting with the new values
new_retrofitted_toy_vecs, new_updates = retrofitting_wordVecs(wordVecMat, neighbors_matrix, alpha=0.1, beta=1.1000000000000003, nb_iter=15)
new_retrofitted_similarity_matrix = generate_cosine_similarity_matrix(new_retrofitted_toy_vecs)
print_vec_similarities(toy_corpus, new_retrofitted_similarity_matrix)

Similarities with "cat":
  - "cat": 1.0000
  - "tiger": 0.5016
  - "computer": 0.5522
  - "keyboard": 0.5654
  - "plane": 0.7554
  - "car": 0.6170
  - "doctor": 0.7334
  - "nurse": 0.8916
  - "love": 0.9205
  - "sex": 0.6468

Similarities with "tiger":
  - "cat": 0.5016
  - "tiger": 1.0000
  - "computer": 0.3162
  - "keyboard": 0.4517
  - "plane": 0.3476
  - "car": 0.3347
  - "doctor": 0.3607
  - "nurse": 0.5070
  - "love": 0.5040
  - "sex": 0.4770

Similarities with "computer":
  - "cat": 0.5522
  - "tiger": 0.3162
  - "computer": 1.0000
  - "keyboard": 0.5762
  - "plane": 0.7584
  - "car": 0.5758
  - "doctor": 0.5341
  - "nurse": 0.4576
  - "love": 0.5711
  - "sex": 0.3314

Similarities with "keyboard":
  - "cat": 0.5654
  - "tiger": 0.4517
  - "computer": 0.5762
  - "keyboard": 1.0000
  - "plane": 0.6427
  - "car": 0.4834
  - "doctor": 0.4639
  - "nurse": 0.5093
  - "love": 0.6120
  - "sex": 0.3910

Similarities with "plane":
  - "cat": 0.7554
  - "tiger": 0.3476
  - "computer": 0.7

In [137]:
# Difference between original and after tuning hyperparam
print_vec_difference(toy_corpus, similarity_matrix, new_retrofitted_similarity_matrix) 


Similarities with "cat":
  - "cat": 1.0000 -> 1.0000 (Difference: 0.0000)
  - "tiger": 0.5173 -> 0.2020 (Difference: -0.3153)
  - "computer": 0.1732 -> 0.1972 (Difference: 0.0239)
  - "keyboard": 0.1834 -> 0.2055 (Difference: 0.0220)
  - "plane": 0.1833 -> 0.3945 (Difference: 0.2111)
  - "car": 0.2153 -> 0.2635 (Difference: 0.0482)
  - "doctor": 0.1292 -> 0.3434 (Difference: 0.2141)
  - "nurse": 0.1594 -> 0.5485 (Difference: 0.3892)
  - "love": 0.1406 -> 0.5941 (Difference: 0.4535)
  - "sex": 0.1368 -> 0.2599 (Difference: 0.1231)

Similarities with "tiger":
  - "cat": 0.5173 -> 0.2020 (Difference: -0.3153)
  - "tiger": 1.0000 -> 1.0000 (Difference: 0.0000)
  - "computer": 0.0677 -> 0.0903 (Difference: 0.0226)
  - "keyboard": 0.0654 -> 0.1831 (Difference: 0.1177)
  - "plane": 0.1660 -> 0.0731 (Difference: -0.0930)
  - "car": 0.1672 -> 0.1003 (Difference: -0.0669)
  - "doctor": 0.0835 -> 0.0891 (Difference: 0.0056)
  - "nurse": 0.1111 -> 0.2026 (Difference: 0.0915)
  - "love": 0.0871 ->

In [138]:
# Difference between retrofitted embeddings and after tuning hyperaparams
print_vec_difference(toy_corpus, retrofitted_similarity_matrix, new_retrofitted_similarity_matrix)


Similarities with "cat":
  - "cat": 1.0000 -> 1.0000 (Difference: -0.0000)
  - "tiger": 0.2028 -> 0.2020 (Difference: -0.0008)
  - "computer": 0.1974 -> 0.1972 (Difference: -0.0002)
  - "keyboard": 0.2057 -> 0.2055 (Difference: -0.0003)
  - "plane": 0.3947 -> 0.3945 (Difference: -0.0002)
  - "car": 0.2636 -> 0.2635 (Difference: -0.0001)
  - "doctor": 0.3436 -> 0.3434 (Difference: -0.0002)
  - "nurse": 0.5487 -> 0.5485 (Difference: -0.0001)
  - "love": 0.5941 -> 0.5941 (Difference: -0.0001)
  - "sex": 0.2601 -> 0.2599 (Difference: -0.0002)

Similarities with "tiger":
  - "cat": 0.2028 -> 0.2020 (Difference: -0.0008)
  - "tiger": 1.0000 -> 1.0000 (Difference: -0.0000)
  - "computer": 0.0903 -> 0.0903 (Difference: 0.0000)
  - "keyboard": 0.1831 -> 0.1831 (Difference: 0.0001)
  - "plane": 0.0734 -> 0.0731 (Difference: -0.0004)
  - "car": 0.1005 -> 0.1003 (Difference: -0.0001)
  - "doctor": 0.0891 -> 0.0891 (Difference: -0.0000)
  - "nurse": 0.2028 -> 0.2026 (Difference: -0.0002)
  - "love

In [156]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(index=wordList, columns=wordList)

# Loop over each word pair and calculate similarity scores
for word1 in wordList:
    for word2 in wordList:
        word1_index = wordList.index(word1)
        word2_index = wordList.index(word2)
        
        # Calculate similarity score after retrofitting
        retrofit_toy_vec, _ = retrofitting_wordVecs(wordVecMat, neighbors_matrix, alpha, beta, nb_iter)
        word1_vec = retrofit_toy_vec[word1_index].reshape(1, -1)
        word2_vec = retrofit_toy_vec[word2_index].reshape(1, -1)
        similarity_after = cosine_similarity(word1_vec, word2_vec)[0, 0]
        
        # Retrieve the evaluation score for the word pair
        score = eval_scores.get((word1, word2))

        # Scale the human score between 0 and 1
        if score is not None:
            scaled_score = score / 10.0
        else:
            scaled_score = None
        
        # Store the scores in the DataFrame
        results_df.loc[word1, word2] = f"Retrofitting: {similarity_after:.2f}, Human: {scaled_score:.2f}" if scaled_score is not None else None

results_df

,cat,tiger,computer,keyboard,plane,car,doctor,nurse,love,sex
cat,None,None,None,None,None,None,None,None,None,None
tiger,"Retrofitting: 0.51, Human: 0.73","Retrofitting: 1.00, Human: 1.00",None,None,None,None,None,None,None,None
computer,None,None,None,"Retrofitting: 0.41, Human: 0.76",None,None,None,None,None,None
keyboard,None,None,None,None,None,None,None,None,None,None
plane,None,None,None,None,None,"Retrofitting: 0.44, Human: 0.58",None,None,None,None
car,None,None,None,None,None,None,None,None,None,None
doctor,None,None,None,None,None,None,None,"Retrofitting: 0.61, Human: 0.70",None,None
nurse,None,None,None,None,None,None,None,None,None,None
love,None,None,None,None,None,None,None,None,None,"Retrofitting: 0.38, Human: 0.68"
sex,None,None,None,None,None,None,None,None,None,None
